<a href="https://colab.research.google.com/github/Raghvv12/India_EV_Analysis/blob/main/AtliQ_Motors_India_EV_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **AtliQ_Motors_India_EV_Analysis**

#### **Objective:**

The objective of this project is to conduct a **comprehensive analysis of the Indian Electric Vehicle (EV) and Hybrid market** to support **AtliQ Motors’ strategic expansion** into India. Despite holding a strong 25% market share in North America, AtliQ Motors currently has less than 2% in India. This study aims to uncover **market opportunities, growth potential, and regional performance patterns** to guide the company’s entry and scaling strategy in India’s EV ecosystem.



The final deliverable will combine **quantitative insights** with **interactive visualizations** to support AtliQ Motors’ leadership in making informed strategic decisions about the Indian EV market.

In [1]:
import pandas as pd

## **Loading the Data**

In [8]:
dim_date = pd.read_csv("/content/dim_date.csv")
sales_makers = pd.read_csv("/content/sales_by_makers.csv")
sales_state = pd.read_csv("/content/sales_by_state.csv")

### **Exploring the data**

In [20]:
print("Sales by Makers:")
print(sales_makers.info())
print(sales_makers.head())

Sales by Makers:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    816 non-null    object 
 1   vehicle_category        816 non-null    object 
 2   maker                   816 non-null    object 
 3   electric_vehicles_sold  816 non-null    float64
dtypes: float64(1), object(3)
memory usage: 25.6+ KB
None
        date vehicle_category         maker  electric_vehicles_sold
0  01-Apr-21       2-Wheelers  OLA ELECTRIC                     0.0
1  01-Apr-22       2-Wheelers      OKAYA EV                     0.0
2  01-May-21       2-Wheelers  OLA ELECTRIC                     0.0
3  01-Jun-21       2-Wheelers  OLA ELECTRIC                     0.0
4  01-Jul-21       2-Wheelers  OLA ELECTRIC                     0.0


In [21]:
print("Sales by State:")
print(sales_state.info())
print(sales_state.head())


Sales by State:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445 entries, 0 to 2444
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    2445 non-null   object
 1   state                   2445 non-null   object
 2   vehicle_category        2445 non-null   object
 3   electric_vehicles_sold  2445 non-null   int64 
 4   total_vehicles_sold     2445 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 95.6+ KB
None
        date   state vehicle_category  electric_vehicles_sold  \
0  01-Apr-21  Sikkim       2-Wheelers                       0   
1  01-Apr-21  Sikkim       4-Wheelers                       0   
2  01-May-21  Sikkim       2-Wheelers                       0   
3  01-May-21  Sikkim       4-Wheelers                       0   
4  01-Jun-21  Sikkim       2-Wheelers                       0   

   total_vehicles_sold  
0                  398  
1                

In [23]:
print("Date Dimension:")
print(dim_date.info())
print(dim_date.head())

Date Dimension:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         36 non-null     object
 1   fiscal_year  36 non-null     int64 
 2   quarter      36 non-null     object
dtypes: int64(1), object(2)
memory usage: 996.0+ bytes
None
        date  fiscal_year quarter
0  01-Apr-21         2022      Q1
1  01-May-21         2022      Q1
2  01-Jun-21         2022      Q1
3  01-Jul-21         2022      Q2
4  01-Aug-21         2022      Q2


In [25]:
print(sales_makers.isnull().sum())
print(sales_state.isnull().sum())
print(dim_date.isnull().sum())

date                      0
vehicle_category          0
maker                     0
electric_vehicles_sold    0
dtype: int64
date                      0
state                     0
vehicle_category          0
electric_vehicles_sold    0
total_vehicles_sold       0
dtype: int64
date           0
fiscal_year    0
quarter        0
dtype: int64


In [26]:
sales_makers = pd.merge(sales_makers, dim_date, on='date', how='left')
sales_state = pd.merge(sales_state, dim_date, on='date', how='left')

**Now both tables will have `fiscal_year` and `quarter` columns, after Merging `dim_date` with both `sales_makers` and `sales_state`**

### **Feature Engineering for Analysis**

In [27]:
# EV Penetration Rate at State Level
sales_state['ev_penetration_rate'] = (sales_state['electric_vehicles_sold'] / sales_state['total_vehicles_sold']) * 100

# Aggregating by Year and Maker
maker_yearly = sales_makers.groupby(['fiscal_year', 'vehicle_category', 'maker'], as_index=False)['electric_vehicles_sold'].sum()

# Aggregating by Year and State
state_yearly = sales_state.groupby(['fiscal_year', 'state', 'vehicle_category'], as_index=False).agg({
    'electric_vehicles_sold': 'sum',
    'total_vehicles_sold': 'sum',
    'ev_penetration_rate': 'mean'
})

### **Identifing Top & Bottom Performers**

In [28]:
# Top 3 & Bottom 3 Makers for FY 2024 (2-Wheelers)
top_bottom_makers_2024 = maker_yearly.query('fiscal_year == 2024 and vehicle_category == "2-Wheelers"')
top_3 = top_bottom_makers_2024.nlargest(3, 'electric_vehicles_sold')
bottom_3 = top_bottom_makers_2024.nsmallest(3, 'electric_vehicles_sold')

# Top 5 States by Penetration (2W and 4W)
top5_states_2w = state_yearly.query('fiscal_year == 2024 and vehicle_category == "2-Wheelers"').nlargest(5, 'ev_penetration_rate')
top5_states_4w = state_yearly.query('fiscal_year == 2024 and vehicle_category == "4-Wheelers"').nlargest(5, 'ev_penetration_rate')

### **Quarterly Trends**

In [29]:
quarterly_trends = sales_makers.groupby(['fiscal_year', 'quarter', 'maker', 'vehicle_category'], as_index=False)['electric_vehicles_sold'].sum()

### **State-wise CAGR Calculation (2022–2024)**

### **Exporting Cleaned and Aggregated CSV**

In [34]:
sales_makers.to_csv('cleaned_sales_maker.csv', index=False)
sales_state.to_csv('cleaned_sales_state.csv', index=False)
state_cagr.to_csv('cleaned_cagr.csv', index=False)